# Introduction to metaprogramming: "Code that creates code" 

Adaptado del [Workshop de David Sanders](http://juliacon.org/workshops.html#invitation), JuliaCon 2016 (Esperar videos!)

Julia has strong **metaprogramming** capabilities. What does this mean?

> **meta**: something on a higher level

**metaprogramming** = "higher-level programming"

i.e. writing code (a program) to manipulate not data, but code (that itself manipulates data)


## A simple example 

This is too hard [**exercise**: write this functionality and make it into a package!], let's start out with a simpler example: Wilkinson-type polynomials.
The [Wilkinson polynomial](https://en.wikipedia.org/wiki/Wilkinson's_polynomial) is

$$p_{20}(x) := (x-1) \cdot (x-2) \cdot \cdots \cdot (x-20) = \prod_{i=1}^{20} (x-i).$$

[Polynomials like this are interesting, since the eigenvalues of the associated "companion matrix", which are used to find the roots of the polynomial, are very sensitive to perturbations of the coefficients of the polynomial.]

Suppose we wish to define this polynomial in Julia. The simple way would be to write it out explicitly:

In [1]:
function poliwilk(n,x)
    resultado = (x-1)
    for i in 2:n
        resultado *= (x-i)
    end
    resultado
end
    

poliwilk (generic function with 1 method)

In [2]:
function p_n(x)
    resultado = (x-1)
    for i in 2:n
        resultado *= (x-i)
    end
    resultado
end
    

p_n (generic function with 1 method)

In [3]:
p_5(2.0)

LoadError: LoadError: UndefVarError: p_5 not defined
while loading In[3], in expression starting on line 1

In [4]:
p_5(x) = (x-1) * (x-2) * (x-3) * (x-4) * (x-5)

p_5 (generic function with 1 method)

## Expressions 

Let's take the simplest polynomial, that we write as `(x-1) * (x-2)`. We can view this as a piece of Julia code, called an **expression**, and can tell Julia this as follows:

In [5]:
ex = quote   
        (x - 1) * (x - 2)
    end

quote  # In[5], line 2:
    (x - 1) * (x - 2)
end

In [6]:
typeof(ex)

Expr

In [7]:
ex(2)

LoadError: LoadError: MethodError: `call` has no method matching call(::Expr, ::Int64)
Closest candidates are:
  BoundsError()
  BoundsError(!Matched::Any...)
  DivideError()
  ...
while loading In[7], in expression starting on line 1

For small pieces of code like this, an alternative syntax uses the `:( ... )`  operator:

In [8]:
ex = :( (x-1) * (x-2) )

:((x - 1) * (x - 2))

What does Julia think this is?

In [9]:
typeof(ex)

Expr

We see that Julia has a type `Expr` that represents an **expression object**, that we can think of as a "Julia code snippet".

We would like to be able to execute this code; we can do so with `eval` ("evaluate"):

In [10]:
eval(ex)

LoadError: LoadError: UndefVarError: x not defined
while loading In[10], in expression starting on line 1

In [11]:
x =2.
ex = :((x-1)*(x-2))
eval(ex)

0.0

We see that `x` is not defined, since Julia is trying to do the same as if we had written the following straight at the prompt

In [12]:
(x-1) * (x-2)

0.0

If we give `x` a value first, then it works:

In [13]:
x = 3.5;
#(x-1) * (x-2)

3.5

And so hence does the `eval`:

In [14]:
eval(ex)

3.75

For example, we can try to write a simple formula evaluator:

In [15]:
formula = "3x"
form = :(3x)

:(3x)

In [16]:
eval(form)

10.5

This does not do what we expect, since we have a string, not a Julia expression:

In [17]:
typeof(formula)

ASCIIString

To convert this string into an expression, we use `parse`:

In [18]:
formula

"3x"

In [19]:
formula2 = parse(formula)

:(3x)

In [20]:
eval(formula2)

10.5

Note that the really hard work, that of parsing the expression, i.e. converting it into an internal representation in terms of a tree, has already been done for us by Julia. It is thus, happily, usually not necessary for us to write our own parser; we just leverage Julia's own! This is one of many ways in which Julia minimises the work that we need to do, compared to other languages.

## Structure of `Expr`essions 

An expression object of type `Expr` has an internal structure that corresponds to the **abstract syntax tree** (AST) of the expression, i.e. a tree, whose nodes are operations, and whose children are subtrees.
We can see this in two ways, using `dump`:

In [21]:
ex

:((x - 1) * (x - 2))

In [22]:
dump(ex)

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol *
    2: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol -
        2: Symbol x
        3: Int64 1
      typ: Any
    3: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol -
        2: Symbol x
        3: Int64 2
      typ: Any
  typ: Any


which shows everything [up to a pre-determined depth] in detail, or 

The point is that since an `Expr`ession is a **standard Julia objects**, we can **use standard Julia commands to manipulate it**! 

As usual, doing `ex.<TAB>` gives a list of the fields in the type `Expr`, or we can use `fieldnames(ex)`:

In [23]:
fieldnames(ex)

3-element Array{Symbol,1}:
 :head
 :args
 :typ 

and then examine the fields:

In [24]:
ex.head

:call

This tells us that the expression represents a function call, whose arguments are the following:

In [25]:
ex.args

3-element Array{Any,1}:
 :*      
 :(x - 1)
 :(x - 2)

[`ex.typ` is not useful for the user.]

The first element of the array `ex.args` is the function to be called:

In [26]:
ex.args[1]

:*

and the second element is

In [27]:
ex.args[2]

:(x - 1)

that is, it is itself an `Expr`ession:

In [28]:
typeof(ans)

Expr

Thus the structure of `Expr`essions is fundamentally recursive, and so lends itself naturally to recursive algorithms.

We can dive, in turn, into the structure of each element:

In [29]:
ex.args[2].args

3-element Array{Any,1}:
  :-
  :x
 1  

In [30]:
ex.args[2].args[2]

:x

What is this `:x`?

In [31]:
typeof(ex.args[2].args[2])

Symbol

We see that it has a special `Symbol` type, which represents the atoms (smallest parts) of an expression.

## Modifying expressions 

Now, what happens if we **modify** this?  Let's make a copy of the expression first so that we can compare the two.

In [32]:
ex2 = copy(ex)

:((x - 1) * (x - 2))

In [33]:
ex2.args[2].args[2] 

:x

In [34]:
ex2.args[2].args[2] = :z

:z

We have changed something inside the object `ex2`, so let's look at it:

In [35]:
ex2

:((z - 1) * (x - 2))

In [36]:
ex

:((x - 1) * (x - 2))

The original expression has, indeed, changed! That is, we have taken a piece of Julia code, and used Julia itself to manipulate it into a different piece of Julia code. This is one of the simplest examples of metaprogramming.

Now we can define `x` and `z` and evaluate the expressions:

In [37]:
x = 3.5; z = 4.5

4.5

In [38]:
eval(ex), eval(ex2)

(3.75,5.25)

## Code generation

Let's return to the original question: how to create a long polynomial expression. 
So far, we have not seen how to *add* code, only *change* code that is already there. 

A natural idea is to build the code up step by step; this is known as **code generation**.

Let's start again from the first term:

In [39]:
ex = :(x-1)

:(x - 1)

In [40]:
typeof(ex)

Expr

We would like to take what we have and create code that is "what we have multiplied by `:(x-2)`". Let's try:

In [41]:
ex_new = :(ex * (x-2))

:(ex * (x - 2))

This doesn't work, since `ex` is treated as a symbol, whereas we need the *value* contained in the *variable* called `ex`. This is obtained using the `$` operator, a procedure called **interpolation**. (Compare this to string interpolation.)

In [42]:
ex = :( $ex * (x-2) )

:((x - 1) * (x - 2))

Now we can continue:

In [43]:
ex = :( $ex * (x-3) ) 

:(((x - 1) * (x - 2)) * (x - 3))

Finally we see how to construct our loop:

In [44]:
n = 10
ex = :(x-1)

for i in 2:n
    ex = :( $ex * (x-i) )
end

In [45]:
ex

:((((((((((x - 1) * (x - i)) * (x - i)) * (x - i)) * (x - i)) * (x - i)) * (x - i)) * (x - i)) * (x - i)) * (x - i))

This did not work, since once again we did not want "the code '`i`'", but rather the value of the variable `i`. So:

In [46]:
n = 10
ex = :(x-1)

for i in 2:n
    ex = :( $ex * (x - $i) )
end

In [47]:
ex

:((((((((((x - 1) * (x - 2)) * (x - 3)) * (x - 4)) * (x - 5)) * (x - 6)) * (x - 7)) * (x - 8)) * (x - 9)) * (x - 10))

This is almost what we would write by hand, except for the large number of parentheses.

Now we need to produce the name of the function:

In [48]:
?symbol

search: symbol Symbol SymbolNode



```
symbol(x...) -> Symbol
```

Create a `Symbol` by concatenating the string representations of the arguments together.


In [49]:
#workspace()

In [50]:
name = symbol("p_", n)   # use `Symbol` instead of `symbol` in Julia v0.5

:p_10

The code is then

In [51]:
code = :( $name(x) = $ex )

:(p_10(x) = begin  # In[51], line 1:
            (((((((((x - 1) * (x - 2)) * (x - 3)) * (x - 4)) * (x - 5)) * (x - 6)) * (x - 7)) * (x - 8)) * (x - 9)) * (x - 10)
        end)

We can wrap this in a function:

In [52]:
?symbol

search: symbol Symbol SymbolNode



```
symbol(x...) -> Symbol
```

Create a `Symbol` by concatenating the string representations of the arguments together.


In [53]:
x =1

1

In [54]:
2*x

2

In [55]:
2x

2

In [ ]:
symbol("p_",5) 

In [59]:
function poliwilk(n,x)
    resultado = (x-1)
    for i in 2:n
        resultado *= (x-i)
    end
    resultado
end

poliwilk (generic function with 1 method)

In [60]:
function make_wilkinson(n)
    
    ex = :(x-1)

    for i in 2:n
        ex = :( $ex * (x - $i) )
    end
    
    name = symbol("p_", n) 
    code = :( $name(x) = $ex )
    
    eval(code)
end

make_wilkinson (generic function with 1 method)

In [62]:
p_10(32.4)

1.8728837025866712e14

In [63]:
poliwilk(10,32.4)

1.8728837025866712e14

Finally we evaluate this:

In [64]:
make_wilkinson(10)

p_10 (generic function with 1 method)

In [65]:
make_wilkinson(100)

p_100 (generic function with 1 method)

This creates a function with the name `p_10` that does what we would write by hand.

In [85]:
@time p_100(0.5)

  0.000005 seconds (5 allocations: 176 bytes)


5.25879029195643e156

In [86]:
t1 = @elapsed p_100(0.5)
t2 = @elapsed poliwilk(100,0.5)
t1/t2

1.3235466823253081

### Starting from empty

In some cases, it is useful to start from something empty and add statements to it.
To do this, we can do, for example,

In [ ]:
code = quote end   # empty

In [ ]:
dump(code)

We can add statements using the standard Julia `push!`:

In [ ]:
Base.push!(code.args, :(a = 1))
Base.push!(code.args, :(b = a + 1))
code

In [ ]:
eval(code)

In [ ]:
@eval $code

**Exercise**: Build up the Wilkinson example using this technique.

## Repetitive code

Code generation is used frequently in Julia code when repetitive code is required. For example, let's return to the idea of wrapping a type:

In [87]:
workspace()

In [88]:
type OurFloat
    x::Float64
end

We can generate objects of this type:

In [89]:
a = OurFloat(3)
b = OurFloat(4)

OurFloat(4.0)

In [91]:
a.x

3.0

But arithmetic operations are not defined:

In [92]:
a + b

LoadError: LoadError: MethodError: `+` has no method matching +(::OurFloat, ::OurFloat)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
while loading In[92], in expression starting on line 1

We can define them in the natural way:

In [96]:
import Base: +, -, *, /
+(a::OurFloat, b::OurFloat) = a.x + b.x
-(a::OurFloat, b::OurFloat) = a.x - b.x

- (generic function with 205 methods)

But this will quickly get dull, and we could easily make a mistake. 
As usual, whenever we are repeating something more than twice, we should try to automate it.

We have some code of the form

    *op*(a::OurFloat, b::OurFloat) = a.x *op* b.x
    
where `*op*` is supposed to represent the operator. Julia allows us to do this almost literally; we just need to substitute in the *value* of the *variable* `op`! This is an **exercise**

In [97]:
for op in (:+, :-, :*, :/)
    #@show op
    ex = :( $(op)(a::OurFloat, b::OurFloat) = $(op)(a.x, b.x) )
    @show ex
end

ex = :(a::OurFloat + b::OurFloat = begin  # In[97], line 3:
            a.x + b.x
        end)
ex = :(a::OurFloat - b::OurFloat = begin  # In[97], line 3:
            a.x - b.x
        end)
ex = :(a::OurFloat * b::OurFloat = begin  # In[97], line 3:
            a.x * b.x
        end)
ex = :(a::OurFloat / b::OurFloat = begin  # In[97], line 3:
            a.x / b.x
        end)


Finally we need to evaluate the code. The combination of `eval` and `:(...)` that we used above can be abbreviated to `@eval`:

In [98]:
for op in (:+, :-, :*, :/)
    #@eval $(op)(a::OurFloat, b::OurFloat) = $(op)(a.x, b.x) 
    eval(:($(op)(a::OurFloat, b::OurFloat) = $(op)(a.x, b.x)))
end

In [99]:
a*b

12.0

In [ ]:
@which a*b